<a href="https://colab.research.google.com/github/2813/ODS-homework/blob/main/03_ODS_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Выбор моделей и работа с признаками**

## **Подготовка для работы в Google Colab или Kaggle**

#### Код для подключения Google Drive в Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Код для получения пути к файлам в Kaggle

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### Код для установки библиотек

In [ ]:
%pip install numpy==1.26.4 pandas==2.1.4 scikit-learn==1.7.0 statsmodels==0.14.4 matplotlib==3.8.0 seaborn==0.13.2  nltk==3.9.1 missingno==0.5.2 mlxtend==0.23.4

## **Важная информация**

**Для правильного воспроизведения результатов** решения задач:

* Рекомендуется придерживаться имеющего в заданиях кода в исходной последовательности. Для этого при решении задач **восстановите недостающие фрагменты кода, которые отмечены символом** `...` (Ellipsis).

* Если класс, функция или метод предусматривает параметр random_state, всегда указывайте **random_state=RANDOM_STATE**.

* Для всех параметров (кроме random_state) класса, функции или метода **используйте значения по умолчанию, если иное не указано в задании**.

**Если скорость обучения слишком низкая**, рекомендуется следующее:

* В модели или/и GridSearchCV поменяйте значение параметра n_jobs, который отвечает за параллелизм вычислений.

* Воспользуйтесь вычислительными ресурсами Google Colab или Kaggle.

***Использовать GPU не рекомендуется, поскольку результаты обучения некоторых моделей могут отличаться на CPU и GPU.***

После выполнения каждого задания **ответьте на вопросы в тесте.**

**ВНИМАНИЕ:** **После каждого нового запуска ноутбука** перед тем, как приступить к выполнению заданий, проверьте настройку виртуального окружения, выполнив код в ячейке ниже.

In [ ]:
# Код для проверки настройки виртуального окружения

import sys
from importlib.metadata import version

required = {
    'python': '3.11.x',
    'numpy': '1.26.4',
    'pandas': '2.1.4',
    'scikit-learn': '1.7.0',
    'statsmodels': '0.14.4',
    'matplotlib': '3.8.0',
    'seaborn': '0.13.2',
    'nltk': '3.9.1',
    'missingno': '0.5.2',
    'mlxtend': '0.23.4'
}

print(f'{"Компонент":<15} | {"Требуется":<12} | {"Установлено":<12} | {"Соответствие"}')
print('-' * 62)

environment_ok = True
for lib, req_ver in required.items():
    try:
        if lib == 'python':
            inst_ver = sys.version.split()[0]
            status = '✓' if sys.version_info.major == 3 and sys.version_info.minor == 11 else f'x (требуется {req_ver})'
        else:
            inst_ver = version(lib)
            if inst_ver == req_ver:
                status = '✓'
            else:
                environment_ok = False
                status = f'x (требуется {req_ver})'
    except:
        environment_ok = False
        inst_ver = '-'
        status = 'x (не установлена)'
    print(f'{lib:<15} | {req_ver:<12} | {inst_ver:<12} | {status}')

print('\nРезультат проверки: ',
      '✓\nВсе версии соответствуют требованиям'
      if environment_ok else
      'x\nВНИМАНИЕ: Версии некоторых компонентов не соответствуют требованиям!\n'
      'Для решения проблемы обратитесь к инструкции по настройке виртуального окружения')

## **Импорт библиотек и вспомогательные функции**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

from mlxtend.evaluate import bias_variance_decomp
from mlxtend.feature_selection import SequentialFeatureSelector

from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.metrics import classification_report, roc_auc_score, f1_score, mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from statsmodels.stats.weightstats import ttest_ind

import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet, stopwords

In [ ]:
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)

In [ ]:
RANDOM_STATE = 42

In [ ]:
def metrics_report(y_true, y_pred):
    """
    Выводит отчёт с основными метриками качества регрессии.
    Округляет до 4-х знаков после запятой и выводит значения R2 (коэффициент детерминации), RMSE (среднеквадратичная ошибка) и MAPE (средняя абсолютная процентная ошибка) для оценки качества предсказаний.

    Аргументы:
        y_true (numpy.ndarray): Истинные значения целевой переменной.
        y_pred (numpy.ndarray): Предсказанные значения целевой переменной.
    """
    print(f'R2 score: {r2_score(y_true, y_pred):.4f}')
    print(f'RMSE: {mean_squared_error(y_true, y_pred)**0.5:.4f}')
    print(f'MAPE: {mean_absolute_percentage_error(y_true, y_pred):.4f}')

### **Обработка пропущенных значений**

**Пропуски в данных (NaN, NULL)** — это незаполненные, пустые ячейки в данных, наличие которых влечет за собой множество проблем, включая искажение данных и снижение качества моделей машинного обучения.

**Методы обработки пропусков:**

* Удаление пропусков:

    * Удаление строк — удаление всех строк, содержащих хотя бы один пропуск.

    * Удаление столбцов — удаление столбцов (переменных), имеющих большое число пропусков (к примеру, более 50%).

* Заполнение константами:

    * Заполнение числовыми данными — 0, -1, 9999.

    * Заполнение категориальными данными — 'NaN', 'Missing'.

* Добавление бинарного признака — индикатора пропуска.

* Предсказание пропусков (импутация, imputation):

    * Заполнение средним/медианой (числовые данные) или модой (категориальные данные).

    * Заполнение случайным значением из распределения.

    * Заполнение с помощью прогноза моделей (KNN, Random Forest, MICE).

Пропуски в данных могут возникать по разным причинам, и от этого зависит, какой метод обработки будет наиболее эффективным. Выделяют **три основных типа пропусков**:

* MCAR (Missing Completely At Random) — пропуски полностью случайны.

    * Удаление строк (если пропусков мало).

    * Заполнение средним/медианой/модой (если пропусков много).

    * Множественная импутация (MICE, KNN, Random Forest).

* MAR (Missing At Random) – пропуски случайны, но зависят от других наблюдаемых данных.

    * Условное заполнение по группам.

    * Добавление бинарного признака.

    * Множественная импутация (MICE, KNN, Random Forest).

* MNAR (Missing Not At Random) – пропуски неслучайны и зависят от ненаблюдаемых факторов.

    * Сбор дополнительных данных (если существует возможность).

    * Модели с учётом MNAR (к примеру, Heckman correction).

    * Множественная импутация с помощью более сложных моделей (к примеру, Deep Learning с учётом пропусков).

*Выше перечислены основные методы обработки пропущенных значений. На практике количество существующих методов больше, а их применение **зависит от задачи***.

### **Датасет *Life Expectancy (WHO)***

**Для решения заданий 1 — 3 рассмотрим датасет [Life Expectancy (WHO)](https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who).**

**ВНИМАНИЕ:** При решении заданий **используйте файл life_expectancy.csv** из приложения к ноутбуку, поскольку исходный датасет был изменен авторами курса.

Рассмотрим набор данных для прогнозирования ожидаемой продолжительности жизни в 193-x странах за несколько лет. Датасет предназначен для выявления ключевых демографических, экономических и социальных факторов, которые в наибольшей степени влияют на показатель продолжительности жизни.

Одной из особенностей датасета является наличие **большого числа пропущенных значений**, которые необходимо обработать.

Целевая переменная — life expectancy (ожидаемая продолжительность жизни в годах).

Датасет содержит 2938 наблюдений за период с 2000 по 2015 год для 193-x стран и 18 признаков, включая демографические показатели, показатели смертности, экономические показатели, показатели здоровья и др.

### ***Задание 1***

Из исходного набора данных удалите строки с пропусками в целевой переменной и разделите датасет на обучающую и тестовую выборки так, чтобы **в тестовую выборку вошли все данные за 2012-2015 год**.

Выполните анализ пропущенных значений **в обучающей выборке**:

1. Посчитайте доли пропусков в каждой переменной с пропусками.

2. Выполните визуальный анализ пропущенных значений с помощью библиотеки missingno: [bar](https://github.com/ResidentMario/missingno?tab=readme-ov-file#bar), [matrix](https://github.com/ResidentMario/missingno?tab=readme-ov-file#matrix), [heatmap](https://github.com/ResidentMario/missingno?tab=readme-ov-file#heatmap).

3. Выполните статистическую проверку пропусков на MCAR (см. далее).

Для статистической проверки пропусков в факторе (predictor) на MCAR (полностью случайные пропуски) предлагается провести t-тест на наличие статистически значимой разницы в среднем значении целевой переменной (life expectancy) в группе с пропущенными значениями и в группе без пропусков.

**Процедура проверки:**

1. Выбирается признак для проверки на MCAR (predictor).

2. Выборка делится на две группы: только с пропущенными значениями predictor и очищенная от всех пропусков predictor.

3. Выполняется t-тест на статистическую разницу средних целевой переменной (feature) в двух группах:

    * $H_0$: Средние значения feature в обеих группах равны. Пропуски в predictor не связаны со значениями feature (MCAR).

    * $H_1$: Средние значения feature в двух группах различаются. Существует связь пропусков в predictor со значениями feature (MAR или MNAR).

Реализуйте предложенную процедуру проверки признаков на MCAR, дополнив функцию mcar_test.

Для каждого признака с пропусками **на обучающей выборке** выполните статистическую проверку пропущенных значений на принадлежность к типу MCAR (функция mcar_test) **на уровне значимости 1%**.

In [ ]:
# Считайте набор данных

df_life = pd.read_csv('life_expectancy.csv')
df_life

In [ ]:
# В методе info отображается количество непустых наблюдений для каждого из признаков

df_life.info()

In [ ]:
# Создайте списки количественных и категориальных переменных (не включая целевую переменную)
# year — количественная переменная

life_num_feat = ...
life_cat_feat = ...

In [ ]:
# Удалите строки с пропусками в целевой переменной

df_life = ...

In [ ]:
# Разделите датасет на обучающую и тестовую выборки так, чтобы в тестовую выборку вошли все данные за 2012-2015 год
# Выделять целевой признак в отдельную переменную не требуется

life_train = ...
life_test = ...

In [ ]:
# Посчитайте доли пропусков в обучающей выборке

life_nan_info = pd.DataFrame({'NaN share': ...})
life_nan_info = life_nan_info[life_nan_info['NaN share'] != 0.0]

#### [bar](https://github.com/ResidentMario/missingno?tab=readme-ov-file#bar)

Визуализация количества отсутствующих данных по столбцам в виде столбчатой диаграммы.

In [ ]:
# Выполните визуальный анализ пропущенных значений в обучающей выборке с помощью msno.bar

...

#### [matrix](https://github.com/ResidentMario/missingno?tab=readme-ov-file#matrix)

Визуализация матрицы пропущенных значений с высокой плотностью. Позволяет выявить закономерности заполненности данных.

In [ ]:
# Выполните визуальный анализ пропущенных значений в обучающей выборке с помощью msno.matrix

...

#### [heatmap](https://github.com/ResidentMario/missingno?tab=readme-ov-file#heatmap)

Тепловая карта корреляции отсутствующих данных. Показывает, насколько сильно наличие или отсутствие одной переменной связано с наличием другой. Корреляция принимает значения от -1 (переменные противоположны в заполненности) до 1 (одинаковая заполненность).

In [ ]:
# Выполните визуальный анализ пропущенных значений в обучающей выборке с помощью msno.heatmap

...

In [ ]:
# Дополните функцию mcar_test

def mcar_test(data, predictor, feature):
    """
    Выполняет статистическую проверку пропущенных значений на принадлежность к типу MCAR.
    Использует t-тест для сравнения распределения feature между группами с пропущенными и существующими значениями predictor.

    Аргументы:
        data (pandas.DataFrame): Датасет с данными для анализа.
        predictor (str): Название столбца в data с пропущенными значениями (целевая переменная для теста MCAR).
        feature (str): Название признака, по которому сравниваются распределения между группами.

    Возвращает:
        float: p-value t-теста.
    """
    data_nans_in_feature = ...      # Все наблюдения в data с пропусками в predictor
    data_non_nans_in_feature = ...  # Все наблюдения в data без пропусков в predictor
    _, p_val, _ = ttest_ind(data_nans_in_feature[feature], data_non_nans_in_feature[feature])
    return p_val

In [ ]:
# Используя функцию mcar_test, выполните проверку пропущенных значений на принадлежность к типу MCAR
# Используйте только обучающую выборку
# Уровень значимости — 1%

p_values = []

...

life_nan_info['p-value'] = p_values

In [ ]:
# Визуализируйте плотность целевой переменной
# График позволяет убедиться, что пропуски типов MAR и MNAR зависят от наблюдаемых данных

predictor = 'BMI'
feature = 'life expectancy'

life_train_nans_in_feature = ...            # Все наблюдения в life_train с пропусками в predictor
life_train_non_nans_in_feature = ...        # Все наблюдения в life_train без пропусков в predictor

dplt = sns.kdeplot(life_train_nans_in_feature[feature], color='red', label=f'В группе с пропусками в {predictor}')
dplt = sns.kdeplot(life_train_non_nans_in_feature[feature], color='blue', label=f'В группе без пропусков в {predictor}')
dplt.set_title(f'Плотность распределения {feature}')
plt.legend()
plt.plot()

### ***Задание 2***

**ВНИМАНИЕ:** Для решения этого задания используйте:

* Обучающую и тестовую выборки из задания 1: `life_train`, `life_test`.

* Датасет с результатами анализа пропущенных значений в обучающей выборке из задания 1: `life_nan_info`.

*Для данного задания предположим, что со временем среднее и дисперсия данных не изменились. Это позволит использовать одни и те же StandardScaler, SimpleImputer и KNNImputer для обучающей и тестовой выборок.*

Выполните предобработку данных (см. код задания).

Используя результаты анализа пропущенных значений в обучающей выборке (`life_nan_info`), выполните обработку пропусков **в обучающей выборке** (`life_train`) по следующим правилам:

* Для всех признаков MCAR (если есть):

    * Заполните пропуски средним, обучив SimpleImputer (`life_mean_imputer`).

* Для всех признаков MAR или MNAR (если есть):

    * Если доля пропусков меньше 5%, заполните пропуски средним, обучив SimpleImputer (`life_mean_imputer`).

    * Если доля пропусков больше 5%, заполните пропуски с помощью метода k-ближайших соседей (KNN, K-Nearest Neighbors), обучив KNNImputer (`life_knn_imputer`).

Заполните пропуски в **тестовой выборке**, используя обученные на обучающей выборке SimpleImputer (`life_mean_imputer`) и KNNImputer (`life_knn_imputer`) для соответствующих признаков.

После обработки пропусков обучите модель линейной регрессии без регуляризации `reg_life` и выведите metrics_report на тестовой выборке.

*Pipeline из библиотеки sklearn позволяет использовать SimpleImputer или KNNImputer как трансформатор, то есть как один из этапов предобработки данных в рамках пайплайна*.

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_life_train, y_life_train = ...
X_life_test, y_life_test = ...

In [ ]:
# Закодируйте категориальные признаки числами 0 и 1 с помощью OneHotEncoder
#   train -> fit_transform
#   test -> transform

life_encoder = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

X_life_train = ...
X_life_test = ...

In [ ]:
# Масштабируйте количественные признаки
#   train -> fit_transform
#   test -> transform

life_scaler = StandardScaler().set_output(transform='pandas')

X_life_train_scaled = ...
X_life_test_scaled = ...

In [ ]:
# Обучите и примените life_mean_imputer (SimpleImputer) к обучающей выборке
#   train -> fit_transform

life_mean_imputer_feat = ...

life_mean_imputer = SimpleImputer(strategy='mean')

X_life_train_scaled_imputed = X_life_train_scaled.copy()
X_life_train_scaled_imputed[life_mean_imputer_feat] = ...

In [ ]:
# Обучите и примените life_knn_imputer (KNNImputer) к обучающей выборке
#   train -> fit_transform

life_knn_imputer_feat = ...

life_knn_imputer = KNNImputer()

X_life_train_scaled_imputed[life_knn_imputer_feat] = ...

In [ ]:
# Примените life_mean_imputer и life_knn_imputer к тестовой выборке
#   test -> transform

X_life_test_scaled_imputed = X_life_test_scaled.copy()
X_life_test_scaled_imputed[life_mean_imputer_feat] = ...
X_life_test_scaled_imputed[life_knn_imputer_feat] = ...

In [ ]:
# Обучите reg_life и выведите metrics_report на тестовой выборке

reg_life = ...
metrics_report(...)

### **Смещение и разброс**

* **Смещение (Bias)** – ожидаемое отклонение предсказаний модели от истинных значений.

* **Разброс (Variance)** – это ошибка, вызванная чувствительностью модели к небольшим изменениям в обучающих данных.

Смещение и разброс отражают два источника ошибки модели, и между ними существует компромисс: снижение смещения часто ведёт к увеличению разброса, и наоборот. Оптимальная модель должна находить баланс между смещением и разбросом, чтобы минимизировать общую ошибку предсказания.

Для анализа ошибки алгоритма применяется **разложение MSE на смещение и разброс (bias-variance decomposition)**.

Пусть $y$ – истинные значения целевой переменой. Мы предполагаем, что существует функция $f(x)=y+e$, где $e$ – ошибка с $\mathbb{E}[e]=0$ (мат. ожидание) и $\mathbb{D}[e]=\sigma^2$ (дисперсия). $\widehat{f}(x)$ – значения целевой переменной, предсказанные моделью.

Тогда

$$\text{MSE}=\mathbb{E}[(y-\widehat{f}(x))^2]=\text{Bias}^2(\widehat{f}(x))+\text{Variance}(\widehat{f}(x))+\sigma^2$$
где
$$\text{Bias}^2(\widehat{f}(x))=(\mathbb{E}[\widehat{f}(x)]-f(x))^2$$
$$\text{Variance}(\widehat{f}(x))=\mathbb{D}[\widehat{f}(x)]=\mathbb{E}[(\mathbb{E}[\widehat{y}]-\widehat{y})^2]$$
$\sigma^2$ — неустранимая ошибка измерения

Подробнее можно изучить по **ссылкам:**

* [Bias-variance decomposition | education.yandex.ru](https://education.yandex.ru/handbook/ml/article/bias-variance-decomposition).

* [Bias-variance decomposition for classification and regression losses | rasbt.github.io](https://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/)

### ***Задание 3***

**ВНИМАНИЕ:** Для решения этого задания используйте:

* Обучающую и тестовую выборки (после заполнения пропусков) из задания 2: `X_life_train_scaled_imputed`, `X_life_test_scaled_imputed`, `y_life_train`, `y_life_test`.

* Обученную модель из задания 2: `reg_life`.

Обучите две модели линейной регрессии с регуляризацией:

* `lasso_life` — регрессия с L1-регуляризацией (LASSO). Оптимальные гиперпараметры обучения подберите с помощью GridSearchCV.

* `ridge_life` — регрессия с L2-регуляризацией (Ridge). Оптимальные гиперпараметры обучения подберите с помощью GridSearchCV.

Разложите среднеквадратическую ошибку (MSE) каждой из трех обученных моделей (`reg_life`, `lasso_life`, `ridge_life`) на компоненты (функция [bias_variance_decomp](https://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/#api)):

* Смещение ($\text{Bias}^2$).

* Разброс ($\text{Variance}$).

In [ ]:
# Выполните разложение MSE модели reg_life на смещение и разброс
# Не забудьте зафиксировать RANDOM_STATE

reg_life_mse, reg_life_bias, reg_life_var = bias_variance_decomp(
    ...
    loss='mse',
    num_rounds=100,
    random_seed=RANDOM_STATE
)

In [ ]:
# Обучите модель lasso_life (LASSO)
# Оптимальные гиперпараметры обучения подберите с помощью GridSearchCV
# Не забудьте зафиксировать RANDOM_STATE

params = {'alpha' : [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0]}
cv = 5

cv_lasso_life = ...
lasso_life = ...

In [ ]:
# Выполните разложение MSE модели lasso_life на смещение и разброс
# Не забудьте зафиксировать RANDOM_STATE

lasso_life_mse, lasso_life_bias, lasso_life_var = bias_variance_decomp(
    ...
    loss='mse',
    num_rounds=100,
    random_seed=RANDOM_STATE
)

In [ ]:
# Обучите модель ridge_life (Ridge)
# Оптимальные гиперпараметры обучения подберите с помощью GridSearchCV
# Не забудьте зафиксировать RANDOM_STATE

params = {'alpha' : [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0]}
cv = 5

cv_ridge_life = ...
ridge_life = ...

In [ ]:
# Выполните разложение MSE модели ridge_life на смещение и разброс
# Не забудьте зафиксировать RANDOM_STATE

ridge_life_mse, ridge_life_bias, ridge_life_var = bias_variance_decomp(
    ...
    loss='mse',
    num_rounds=100,
    random_seed=RANDOM_STATE
)

## **Извлечение признаков из текста**

**Классификация тональности (sentiment analysis)** — это задача определения эмоциональной окраски текста в рамках NLP (Natural Language Processing).

Как правило, тональность текста делят на три категории:

* Позитивная (к примеру, положительный отзыв).

* Нейтральная (информация без явной эмоциональной окраски).

* Негативная (к примеру, критика или жалоба).

Перед обучением моделей необходимо извлечь признаки из текстовых данных, проведя предобработку текста.

Основные этапы предобработки текста:

1. Очистка текста от лишних элементов (не влияющих на тональность):

    * Удаление спецсимволов — удаление HTML-тегов, URL, email, хештегов, упоминаний (@user).

    * Удаление цифр — удаление или замена на числительные.

    * Удаление пунктуации.

    * Удаление лишних пробелов и дублирующихся  символов.

2. Токенизация — разбиение текста на отдельные токены (слова, части слов или символы). Пример: 'Я люблю этот фильм!' $\to$ ['Я', 'люблю', 'этот', 'фильм', '!'].

3. Нормализация слов:

    * Лемматизация — приведение слова к лемме — её нормальной (словарной) форме. Пример: 'бежал' $\to$ 'бежать'.

    * Стемминг — выделение основы слова. Пример: 'бежал' $\to$ 'беж'.

4. Удаление стоп-слов — исключение слов, которые часто встречаются в тексте, но не несут значимой смысловой нагрузки для анализа тональности: местоимения, предлоги, союзы, частицы, артикли (для английского) и т.д.

5. Векторизация текста — преобразование текста в числовой формат для обучения и использования моделей машинного обучения:

    * BoW (Bag-of-words).

    * TF-IDF (Term Frequency — Inverse Document Frequency).

    * Векторные эмбеддинги: FastText, Word2Vec, GloVe.

    * Контекстные эмбеддинги: BERT.

*Это лишь основные этапы предобработки текста. На практике количество существующих методов обработки данных больше, а структура и порядок этапов **зависит от задачи***.

Подробнее можно изучить по **ссылкам:**

* [Основы Natural Language Processing для текста | habr.com](https://habr.com/ru/companies/Voximplant/articles/446738/)

* [Преобразование текстовых данных и работа с ними в Python | education.yandex.ru](https://education.yandex.ru/handbook/data-analysis/article/preobrazovanie-tekstovyh-dannyh-i-rabota-s-nimi-v-python)

* [Краткий обзор техник векторизации в NLP | habr.com](https://habr.com/ru/articles/778048/)

* [How to Create Bert Vector Embeddings? A Comprehensive Tutorial | airbyte.com](https://airbyte.com/data-engineering-resources/bert-vector-embedding)

### **TF-IDF**

**TF-IDF** (Term Frequency — Inverse Document Frequency) — это статистическая мера, для оценки важности слова в контексте документа (набора текстовых данных), являющегося частью коллекции или корпуса документов. Высокий вес по TF-IDF получают слова, которые часто встречаются в конкретном документе, но при этом редко встречаются во всех остальных документах корпуса.

Эта мера состоит из двух компонентов:

* TF (Term Frequency — частота слова). TF измеряет, насколько часто слово встречается в конкретном документе. Чем чаще слово появляется в тексте, тем выше его TF.

$$\text{TF} = \frac{\text{Количество раз, когда слово встретилось в документе}}{\text{Общее количество слов в документе}}$$

* IDF (Inverse Document Frequency — обратная документная частота). IDF измеряет уникальность или информативность слова в масштабах всего корпуса документов. Компонент IDF уменьшает вес слов, которые встречаются слишком часто во многих документах (например, предлоги, союзы). Если слово редкое, его IDF будет высоким.

$$\text{IDF} = \log(\frac{\text{Общее количество документов в корпусе}}{\text{Количество документов, в которых встретилось слово} + 1})$$

Итоговая формула TF-IDF:

$$\text{TF-IDF}=\text{TF} \times \text{IDF}$$

Таким образом, слово получает высокий вес TF-IDF, если оно:

* Часто встречается в данном документе (высокий TF).

* Редко встречается в других документах корпуса (высокий IDF).

TF-IDF позволяет преобразовать наборы текстовых данных в разреженную матрицу признаков, которая может быть использована в алгоритмах машинного обучения.

**Процедура преобразования:**

1. Создание словаря. Составляется полный список всех уникальных слов, которые встретились во всех текстах из корпуса. Каждому уникальному слову присваивается индекс (номер столбца в матрице).

2. Для каждого документа (строки) вычисляется TF-IDF вес каждого слова из словаря. Если слово из общего словаря отсутствует в данном документе, его вес для данного документа будет равен 0.

Итоговая матрица признаков, используемая для обучения моделей ML:

* Строки — документы.

* Столбцы — уникальные слова из всего корпуса документов.

* Значения в ячейках — TF-IDF веса слов.

### ***Задание 4***

Рассмотрим упрощенный пример бинарной классификации тональности текста с помощью логистической регрессии.

Выполните предобработку текстов:

1. Приведите все тексты к нижнему регистру (lowercase). *Подсказка: изучите [векторизованные строковые функции pandas](https://pandas.pydata.org/docs/user_guide/text.html#method-summary).*

2. Очистите тексты от ссылок и лишних символов с помощью функции regex_clean.

3. Дополните функцию `tokenize` и токенизируйте тексты.

4. Дополните функцию `pos_lemmatize` и лемматизируйте тексты c учетом частей речи слов (POS — Part Of Speech). *Подробнее по ссылке: [Lemmatization in NLP | medium.com](https://medium.com/@kevinnjagi83/lemmatization-in-nlp-2a61012c5d66).*

5. Дополните функцию `remove_stop_words` и удалите из текстов стоп-слова.

6. Соберите список лемм (после удаления стоп-слов) в одну строку, отделив леммы знаком пробела. *Подсказка: изучите [векторизованные строковые функции pandas](https://pandas.pydata.org/docs/user_guide/text.html#method-summary).*

7. Разделите датасет на обучающую (60%) и тестовую (40%) выборки со стратификацией по целевой переменной.

Постройте пайплайн `twit_pipeline`:

1. 'tfidf': [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

2. 'clf': LogisticRegression.

Используя пайплайн `twit_pipeline`, подберите оптимальные гиперпараметры обучения с помощью GridSearchCV **(метрика оптимизации — AUC)**, и с оптимальными гиперпараметрами обучите модель `lr_twit` для бинарной классификации тональности текстов.

Для модели `lr_twit` на тестовой выборке постройте отчет по метрикам классификации и посчитайте метрику AUC.

In [ ]:
# Считайте данные

df_twit = pd.read_csv('tweets.csv')
df_twit

In [ ]:
# Удалите из датасета нейтральную ('neutral') тональность

df_twit = df_twit[df_twit['sentiment'] != 'neutral']

In [ ]:
# Закодируйте целевую переменную:
#   0 — 'negative'
#   1 — 'positive'
# Сбросьте индексы в новом датасете

df_twit['sentiment'] = ...
df_twit = df_twit.reset_index(drop=True)

In [ ]:
# Выведите исходный текст с индексом 1901

print(df_twit.iloc[1901]['text'])

In [ ]:
# Приведите все тексты к нижнему регистру (lowercase)

df_twit['text lowercase'] = ...

In [ ]:
# Выведите текст с индексом 1901 после приведения к нижнему регистру

...

In [ ]:
def regex_clean(text):
    """
    Очищает текст от ссылок и лишних символов с помощью регулярных выражений.

    Аргументы:
        text (str): Входной текст для очистки.

    Возвращает:
        str: Очищенный текст.
    """
    # Замена всех ссылок в тексте на пробел
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)

    # Удаление всех цифр, спецсимволов и знаков пунктуации
    text = re.sub('[0-9!#()$\,\'\-\.*+/:;<=>?@[\]^_`{|}\"]+', ' ', text)

    # Удаление лишних пробелов
    text = re.sub('\s+', ' ', text)

    return text

In [ ]:
# Очистите тексты от ссылок и лишних символов с помощью функции regex_clean

df_twit['text cleaned'] = ...

In [ ]:
# Выведите текст с индексом 1901 после удаления ссылок и лишних символов

...

In [ ]:
# Дополните функцию tokenize

def tokenize(text):
    """
    Токенизирует текст на отдельные слова или токены с использованием NLTK.
    Для токенизации используется функиця word_tokenize.

    Аргументы:
        text (str): Входной текст для токенизации.

    Возвращает:
        list(str): Список токенов, полученных в результате разбиения текста.
    """
    return ...

In [ ]:
# Токенизируйте тексты с помощью tokenize

df_twit['text tokenized'] = ...

In [ ]:
# Выведите текст с индексом 1901 после токенизации

...

In [ ]:
def treebank_to_wordnet(treebank_pos_tag):
    """
    Преобразует POS-теги из формата Penn Treebank в формат WordNet.

    Аргументы:
        treebank_pos_tag (str): POS-тег в формате Penn Treebank.

    Возвращает:
        object: POS-тег в формате WordNet. В случае неизвестного тега по умолчанию возвращает wordnet.NOUN (существительное).
    """
    if treebank_pos_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_pos_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_pos_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
# Дополните функцию pos_lemmatize

def pos_lemmatize(tokens):
    """
    Лемматизирует список токенов, преобразуя слова к их базовой форме с учетом части речи (POS).
    Для лемматизации используется WordNetLemmatizer.

    Аргументы:
        tokens (list(str)): Список токенов для лемматизации.

    Возвращает:
        list(str): Список лемм, полученных в результате лемматизации токенов.
    """
    lemmatizer = WordNetLemmatizer()

    # Определение POS-тегов в формате Penn Treebank для каждого токена
    pos_tokens = pos_tag(...)

    # Лемматизируйте каждый токен с учетом его POS-тега
    # POS-теги в формате Penn Treebank необходимо преобразовать в формат WordNet (используйте функцию treebank_to_wordnet)
    lemmas = [lemmatizer.lemmatize(...) for token, pos in ...]
    return lemmas

In [ ]:
# Лемматизируйте тексты с помощью pos_lemmatize

df_twit['text lemmatized'] = ...

In [ ]:
# Выведите текст с индексом 1901 после лемматизации

...

In [ ]:
# Дополните функцию remove_stop_words

def remove_stop_words(lemmas, stop_words):
    """
    Удаляет стоп-слова из списка лемм.

    Аргументы:
        lemmas (list(str)): Список лемм.
        stop_words (list(str)): Список стоп-слов для фильтрации.

    Возвращает:
        list(str): Список лемм после удаления стоп-слов.
    """
    lemmas = ...
    return lemmas

In [ ]:
# Удалите стоп-слова из текстов с помощью remove_stop_words
# В качестве списка стоп-слов используйте список из библиотеки NLTK

stop_words = stopwords.words('english')
df_twit['text without stop words'] = ...

In [ ]:
# Соберите список лемм в одну строку, отделив леммы знаком пробела

df_twit['text preprocessed'] = ...

In [ ]:
# Выведите текст с индексом 1901 после удаления стоп-слов

...

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_twit, y_twit = df_twit['text preprocessed'], df_twit['sentiment']

In [ ]:
# Разделите датасет на обучающую (60%) и тестовую (40%) выборки со стратификацией по целевой переменной
# Не забудьте зафиксировать RANDOM_STATE

X_twit_train, X_twit_test, y_twit_train, y_twit_test = ...

In [ ]:
# Постройте пайплайн twit_pipeline
# Не забудьте зафиксировать RANDOM_STATE

twit_pipeline = Pipeline([
    ('tfidf', ...),
    ('clf', ...)
])

In [ ]:
# Обучите модель lr_twit на обучающих данных
# Оптимальные гиперпараметры обучения подберите с помощью GridSearchCV
# Не забудьте зафиксировать RANDOM_STATE

params = {
    'tfidf__ngram_range': [(1, 1), (1, 2)], # (1, 1) — только отдельные слова / (1, 2) — слова и биграммамы (пары слов)
    'tfidf__max_df': [0.9, 0.95],           # Верхний порог частоты слова в корпусе текстов (если частота выше, слово игнорируется)
    'clf__C': [0.001, 0.1, 1.0, 10.0]       # Параметр регуляризации для логистической регрессии
}
scoring = 'roc_auc'
cv = 5

cv_lr_twit = GridSearchCV(
    estimator=...,
    param_grid=...,
    scoring=...,
    cv=...,
    n_jobs=-1                               # Используйте все доступные ядра CPU
).fit(...)

lr_twit = ...

In [ ]:
cv_lr_twit.best_params_

In [ ]:
# Постройте отчет по метрикам классификации на тестовой выборке для lr_twit

...

In [ ]:
# Рассчитайте метрику AUC на тестовой выборке для lr_twit

...

### **Извлечение признаков из временных рядов**

Целью извлечения признаков из временных рядов является преобразование исходного временного ряда (последовательности точек во времени) в набор информативных и компактных статических признаков.

**Основные типы временных признаков:**

* Признаки на основе лагов (предыдущих значений ряда).

    * Лаговые признаки — это значения временного ряда из предыдущих моментов времени.

    * Скользящее среднее (moving average) — среднее значение за несколько предыдущих периодов.

* Признаки на основе даты и времени.

    * Временные компоненты: час, день недели, день месяца, месяц, год и т.д.

    * Бинарные флаги: является ли день выходным, праздничным и т.д.

* Циклические признаки. Чтобы сохранить информацию о цикличности, признаки преобразуют в двумерное пространство с помощью синуса и косинуса. Для признака со значением $x$ и периодом $T$ (например, для часа $T = 24$, для дня недели $T = 7$) создаются два новых признака:

    $$x_{\text{cos}} = \text{cos}(\frac{2 \pi x}{T})$$

    $$x_{\text{sin}} = \text{sin}(\frac{2 \pi x}{T})$$

*Это лишь основные и наиболее простые типы временных признаков. На практике их количество больше, а использование того или иного типа **зависит от решаемой задачи***.

### **Кросс-валидация временных рядов**

Классические методы кросс-валидации не могут быть использованы для временных рядов, упорядоченных во времени. Это связано с проблемой утечки данных (data leakage): случайное перемешивание временного ряда приведет к тому, что модель будет обучаться на данных из будущего, чтобы предсказать прошлое.

Для временных рядов необходимы особые стратегии кросс-валидации, которые сохраняют временной порядок данных.

**[TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)** — это итератор кросс-валидации временных рядов из библиотеки scikit-learn. TimeSeriesSplit реализует подход, известный как кросс-валидация на расширяющемся окне (expanding window cross-validation): c каждой новой итерацией (фолдом) обучающая выборка увеличивается, а валидационная сдвигается вперед во времени.

TimeSeriesSplit может быть использован для подбора гиперпараметров с помощью GridSearchCV. Если **в качестве параметра cv** в GridSearchCV использовать разбиение временного ряда, полученное с помощью TimeSeriesSplit, GridSearchCV будет обучать и валидировать модель на соответствующих разбиениях (фолдах), обеспечивая корректную оценку качества модели.

### **Датасет *Hourly Energy Consumption***

**Для решения задания 5 рассмотрим датасет [Hourly Energy Consumption](https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption).**

Набор данных предназначен для анализа и прогнозирования почасового потребления электроэнергии в мегаваттах (MW) в различных регионах, входящих в PJM Interconnection LLC — региональную организацию по передаче электроэнергии в восточной части США.

Целевая переменная — AEP_MW (количество потребляемой электроэнергии в мегаваттах за каждый час).

Единственная известная переменная датасета — Datetime (временная отметка с точностью до часа, которая указывает дату и время записи).

### ***Задание 5***

Используя значения ряда (AEP_MW), создайте признаки:

* Лаговые признаки (*подсказка: используйте [shift](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html)*):

    * lag 1h — значение AEP_MW 1 час назад (временной лаг в 1 наблюдение).

    * lag 24h — значение AEP_MW 24 часа назад (временной лаг в 24 наблюдения).

* Скользящее среднее (*подсказка: используйте признак **lag 1h** (скользящее среднее строится по **предыдущим** наблюдениям и не должно включать значение целевой переменной на момент прогноза) и [rolling](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html)*):

    * mean 24h — среднее AEP_MW за **предыдущие** 24 часа (скользящее среднее по 24 **предыдущим** наблюдениям).

    * mean 7d — среднее AEP_MW за **предыдущие** 7 дней (скользящее среднее по 24 $\times$ 7 **предыдущим** наблюдениям).

Используя метку времени (Datetime), создайте признаки:

* Временные компоненты:

    * hour: час.

    * weekday: день недели.

    * day: число месяца.

    * month: месяц.

    * year: год.

* Бинарные флаги:

    * is_weekend: метка выходного дня (суббота и воскресенье).

* Циклические признаки, закодированные с помощью косинуса и синуса:

    * hour_cos, hour_sin: cos и sin от hour ($T = 24$).

    * day_cos, day_sin: cos и sin от day ($T = 30$).

    * weekday_cos, weekday_sin: cos и sin от weekday ($T = 7$).

    * month_cos, month_sin: cos и sin от month ($T = 12$).

Разделите датасет на обучающую и тестовую выборки так, чтобы в обучающую выборку вошли все данные ранее 2017 года, в тестовую — все данные за 2017 год и позже.

Масштабируйте (стандартизируйте) все признаки на обучающей и тестовой выборке.

**На обучающей выборке** подберите оптимальные гиперпараметры обучения линейной регрессии с L2-регуляризацией (Ridge) с помощью **кросс-валидации временных рядов** (TimeSeriesSplit и GridSearchCV), метрика оптимизации — 'neg_mean_squared_error' (**отрицательный MSE**). Рассчитайте лучшее среднее значение **RMSE** (Root Mean Square Error, среднеквадратическая ошибка) по результатам кросс-валидации.

Обучите модель Ridge `ridge_aep` с оптимальными гиперпараметрами на всей обучающей выборке и выведите metrics_report **на тестовой выборке**.

*Для данного задания предположим, что со временем среднее и дисперсия данных не изменились. Это позволит использовать один StandardScaler для обучающей и тестовой выборок.*

In [ ]:
# Считайте набор данных

df_aep = pd.read_csv('AEP_hourly.csv')
df_aep

In [ ]:
# Признак Datetime имеет тип данных object
# Необходимо изменить тип данных для Datetime

df_aep.info()

In [ ]:
# Измените тип Datetime на datetime64[ns]

df_aep['Datetime'] = ...

In [ ]:
# Убедитесь, что тип данных Datetime — datetime64[ns]

df_aep.info()

In [ ]:
# Постройте гистограмму потребления электроэнергии

df_aep['AEP_MW'].hist(bins=100)
plt.xlabel('Почасовое потребление электроэнергии, МВт')
plt.show()

In [ ]:
# Постройте график потребления электроэнергии

df_aep.plot(x='Datetime', y='AEP_MW', figsize=(12, 6))
plt.xlabel('Время')
plt.ylabel('Почасовое потребление электроэнергии, МВт')
plt.show()

In [ ]:
# Создайте признаки lag 1h и lag 24h
# Подсказка: используйте shift

df_aep['lag 1h'] = ...
df_aep['lag 24h'] = ...

In [ ]:
# Создайте признаки mean 24h и mean 7d
# Подсказка: используйте признак lag 1h и rolling

df_aep['mean 24h'] = ...
df_aep['mean 7d'] = ...

In [ ]:
# Закодируйте метку времени (Datetime) как временные компоненты

df_aep['hour'] = ...
df_aep['weekday'] = ...
df_aep['month'] = ...
df_aep['day'] = ...
df_aep['year'] = ...

In [ ]:
# Закодируйте метку времени (Datetime) как бинарный флаг is_weekend

df_aep['is_weekend'] = ...

In [ ]:
# Закодируйте циклические переменные с помощью косинуса и синуса

df_aep['hour_cos'] = ...
df_aep['hour_sin'] = ...

df_aep['day_cos'] = ...
df_aep['day_sin'] = ...

df_aep['weekday_cos'] = ...
df_aep['weekday_sin'] = ...

df_aep['month_cos'] = ...
df_aep['month_sin'] = ...

In [ ]:
# Датасет после создания признаков

df_aep

In [ ]:
# Удалите строки с пропущенными значениями
# Пропущенные значения появились после создания лаговых признаков и скользящих средних

df_aep = ...

In [ ]:
# Выделите объясняемый фактор в отдельную переменную
# Также удалите метку времени (Datetime) из объясняющих переменных

X_aep, y_aep = ...

In [ ]:
# Разделите датасет на обучающую и тестовую выборки:
#   Обучающая выборка — все данные ранее 2017 года
#   Тестовая выборка  — все данные за 2017 год и позже

X_aep_train = ...
y_aep_train = ...

X_aep_test = ...
y_aep_test = ...

In [ ]:
# Масштабируйте все признаки
#   train -> fit_transform
#   test -> transform

aep_scaler = StandardScaler().set_output(transform='pandas')

X_aep_train_scaled = ...
X_aep_test_scaled = ...

In [ ]:
# На обучающей выборке подберите оптимальные гиперпараметры обучения Ridge с помощью кросс-валидации временных рядов
# Используйте TimeSeriesSplit и GridSearchCV
# Не забудьте зафиксировать RANDOM_STATE

params = {
    'alpha': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0]
}
scoring = 'neg_mean_squared_error'
cv = 10

tscv = TimeSeriesSplit(...)

cv_ridge_aep = GridSearchCV(
    estimator=...,
    param_grid=...,
    scoring=...,
    cv=...,
    n_jobs=-1
).fit(...)

In [ ]:
# Рассчитайте лучшее среднее значение RMSE по результатам кросс-валидации

...

In [ ]:
# Обучите ridge_aep с оптимальными гиперпараметрами на всей обучающей выборке и выведите metrics_report на тестовой выборке

ridge_aep = ...
metrics_report(...)

### **Отбор признаков**

**Отбор признаков** (feature selection) — это процесс выбора наиболее информативных переменных из исходного набора данных. Такой подход позволяет упростить модель, сократить время обучения, повысить точность и уменьшить переобучение. Существует три основные группы методов: фильтры, обёртки и встроенные методы.

**Фильтры (filter methods)**

Фильтры оценивают важность признаков на основе статистических критериев независимо от используемой модели машинного обучения. Признаки оцениваются по отдельности, а затем выбирается подмножество признаков, которые лучше всего коррелируют с целевой переменной.

Такие методы просты в реализации и быстры, но не учитывают взаимодействие между признаками и специфику модели.

Примеры методов:

* Корреляционный анализ.

* Критерий $\chi^2$ (хи-квадрат).

* ANOVA F-тест.

**Обёртки (wrapper methods)**

Обёртки используют модели для оценки качества различных подмножеств признаков. Эти методы перебирают различные комбинации признаков, обучают модель и сравнивают качество прогнозов.

Такие методы способны учитывать взаимодействие признаков и специфику модели, но вычислительно затратны.

Примеры методов:

* SFS (Sequential Forward Selection). Отбор начинается с пустого множества признаков. На каждом шаге к текущему набору добавляется тот признак, который при включении даёт наибольшее улучшение качества модели. Процесс продолжается до тех пор, когда не будет достигнут заданный размер подмножества или когда дальнейшее добавление признаков не будет улучшать качество.

* SBS (Sequential Backward Selection). Отбор начинается со всех признаков. На каждом шаге удаляется тот признак, чьё исключение меньше всего ухудшает качество модели. Процесс повторяется, пока не останется заданное число признаков.

* RFE (Recursive Feature Elimination). Отбор осуществляется на основе коэффициентов или важности признаков в обученной модели путем удаления наименее значимого признака. Процесс повторяется рекурсивно, пока не останется заданное количество признаков.

**Встроенные методы (embedded methods)**

Встроенные методы производят отбор признаков непосредственно в процессе обучения модели: используются внутренние механизмы модели для определения важности признаков.

Встроенные методы сочетают преимущества фильтрующих (скорость) и оберточных (учет специфики модели) методов.

Примеры методов:

* L1-регуляризация (LASSO) для линейных моделей.

* Feature Importance в деревьях решений и ансамблевых методах (Random Forest, Gradient Boosting).

*На практике перечисленные методы могут комбинироваться. Например, сначала с помощью фильтра убираются малоинформативные признаки, а затем применяется обёртка для более точного отбора признаков.*

Подробнее можно изучить по **ссылкам:**

* [Comprehensive Guide on Feature Selection | kaggle.com](https://www.kaggle.com/code/prashant111/comprehensive-guide-on-feature-selection)

* [Отбор признаков (Feature selection) | scikit-learn.ru](https://scikit-learn.ru/stable/modules/feature_selection.html#univariate-feature-selection)

### **Датасет *Company Bankruptcy Prediction***

**Для решения задания 6 рассмотрим датасет [Company Bankruptcy Prediction](https://www.kaggle.com/datasets/fedesoriano/company-bankruptcy-prediction).**

**ВНИМАНИЕ:** При решении задания **используйте файл bankruptcy.csv** из приложения к ноутбуку, поскольку исходный датасет был изменен авторами курса.

Набор данных предназначен для выявления компаний с высоким риском банкротства на основе их финансовых показателей. Данные были собраны из базы данных компании Taiwan Economic Journal за период с 1999 по 2009 годы.

Целевая переменная — Bankrupt? (банкротство компании):

0 — в течение рассматриваемого периода компания работала без признаков банкротства.

1 — компания обанкротилась в течение рассматриваемого периода.

В датасете содержатся 95 различных показателей финансового состояния компаний, значения которых были предварительно стандартизированы.

Одной из особенностей набора данных является дисбаланс классов в целевой переменной: только 3.22% от числа компаний в выборке были признаны банкротами за указанный период.

### ***Задание 6***

Используя все признаки, обучите baseline модель логистической регрессии (LogisticRegression) `lr_bankr_baseline` с параметрами:

* solver='liblinear' — алгоритм оптимизации, который поддерживает L1 и L2 регуляризации.

* class_weight='balanced' — корректировка веса классов обратно пропорционально их частотам в обучающем наборе данных.

* random_state=RANDOM_STATE.

*Baseline — это базовая модель, которая используется как отправная точка для оценки и сравнения более продвинутых моделей в задачах машинного обучения.*

Выполните отбор признаков тремя методами и обучите модели на оптимальном наборе признаков:

* **Фильтр: ANOVA F-тест (SelectKBest).**

    1. Постройте пайплайн `bankr_kbest_pipeline`:

        1. 'selector': [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([f_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html)) — выбирает k лучших признаков на основе оценки ANOVA F-теста.

        2. 'clf': LogisticRegression с теми же параметрами, что и baseline модель.

    2. С помощью GridSearchCV и пайплайна `bankr_kbest_pipeline` подберите на обучающей выборке оптимальное количество признаков $k_{\text{KBest}}$, где $k_{\text{KBest}}$ — число от 1 до 40.

    3. На оптимальном наборе из $k_{\text{KBest}}$ признаков на всей обучающей выборке обучите модель (пайплайн) `lr_bankr_kbest` с теми же параметрами, что и baseline модель.

* **Обёртка: SFS (SequentialFeatureSelector).**

    1. Используя [SequentialFeatureSelector](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.feature_selection/#sequentialfeatureselector) из библиотеки mlxtend, подберите на обучающей выборке оптимальный набор из $k_{\text{SFS}}$ признаков, где $k_{\text{SFS}}$ — число от 1 до 40.

    2. На оптимальном наборе из $k_{\text{SFS}}$ признаков на всей обучающей выборке обучите модель `lr_bankr_sfs` с теми же параметрами, что и baseline модель.

* **Встроенный метод: L1 регуляризация.**

    1. С помощью GridSearchCV подберите на обучающей выборке значение оптимального гиперпараметра регуляризации C для **логистической регрессии с L1 регуляризацией** (остальные параметры те же, что и в baseline модели).

    2. На всей обучающей выборке обучите модель с L1 регуляризацией `lr_bankr_l1`, в качестве параметров используя оптимальный гиперпараметр регуляризации и те же параметры, что и в baseline модели.

Для обученных моделей `lr_bankr_kbest`, `lr_bankr_sfs` и `lr_bankr_l1` определите количество отобранных признаков k.

На тестовой выборке для всех моделей, включая baseline (`lr_bankr`, `lr_bankr_kbest`, `lr_bankr_sfs` и `lr_bankr_l1`), постройте  отчёты по метрикам классификации, рассчитайте метрики f1 и AUC.

In [ ]:
# Считайте данные

df_bankr = pd.read_csv('bankruptcy.csv')
df_bankr

In [ ]:
# Всего в датасете 95 признаков (без учета целевой переменной)
# Пропущенные значения отсутствуют

df_bankr.info()

In [ ]:
# В датасете присутствует дисбаланс классов

df_bankr['Bankrupt?'].value_counts(normalize=True)

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_bankr, y_bankr = ...

In [ ]:
# Разделите датасет на обучающую (60%) и тестовую (40%) выборки со стратификацией по целевой переменной
# Не забудьте зафиксировать RANDOM_STATE

X_bankr_train, X_bankr_test, y_bankr_train, y_bankr_test = ...

In [ ]:
# Обучите baseline модель lr_bankr_baseline
# Не забудьте зафиксировать RANDOM_STATE

lr_bankr_baseline = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear', class_weight='balanced')
...

In [ ]:
# Постройте отчет по метрикам классификации для модели lr_bankr на тестовой выборке

...

In [ ]:
# Посчитайте f1 для модели lr_bankr_baseline на тестовой выборке

...

In [ ]:
# Посчитайте AUC для модели lr_bankr_baseline на тестовой выборке

...

In [ ]:
# Постройте пайплайн bankr_kbest_pipeline (см. задание)
# Не забудьте зафиксировать RANDOM_STATE

bankr_kbest_pipeline = Pipeline([
    ('selector', ...),
    ('clf', ...)
])

In [ ]:
# С помощью GridSearchCV и bankr_kbest_pipeline подберите на обучающей выборке оптимальное количество признаков

params = {
    ...: range(1, 40)
}
scoring = 'f1'
cv = 5

cv_lr_bankr_kbest = GridSearchCV(
    estimator=...,
    param_grid=...,
    scoring=...,
    cv=...,
    n_jobs=-1
).fit(...)

In [ ]:
# Выведите оптимальное количество признаков для bankr_kbest_pipeline

...

In [ ]:
# На наборе из оптимального числа признаков на всей обучающей выборке обучите модель lr_bankr_kbest

lr_bankr_kbest = ...

In [ ]:
# Постройте отчет по метрикам классификации для модели lr_bankr_kbest на тестовой выборке

...

In [ ]:
# Посчитайте f1 для модели lr_bankr_kbest на тестовой выборке

...

In [ ]:
# Посчитайте AUC для модели lr_bankr_kbest на тестовой выборке

...

In [ ]:
# Используя SequentialFeatureSelector, подберите на обучающей выборке оптимальный набор признаков
# Не забудьте зафиксировать RANDOM_STATE

bankr_sfs = SequentialFeatureSelector(
    estimator=...,
    k_features=...,
    forward=True,
    floating=False,
    scoring='f1',
    cv=5,
    n_jobs=-1
).fit(...)

In [ ]:
# Выведите оптимальное количество признаков после SFS

...

In [ ]:
# Создайте новые выборки с оптимальными признаками после SFS

X_bankr_train_sfs = ...
X_bankr_test_sfs = ...

In [ ]:
# На оптимальном наборе признаков на всей обучающей выборке обучите модель lr_bankr_sfs
# Не забудьте зафиксировать RANDOM_STATE

lr_bankr_sfs = ...

In [ ]:
# Постройте отчет по метрикам классификации для модели lr_bankr_sfs на тестовой выборке

...

In [ ]:
# Посчитайте f1 для модели lr_bankr_sfs на тестовой выборке

...

In [ ]:
# Посчитайте AUC для модели lr_bankr_sfs на тестовой выборке

...

In [ ]:
# С помощью GridSearchCV подберите на обучающей выборке значение оптимального гиперпараметра регуляризации C
# Не забудьте зафиксировать RANDOM_STATE

estimator = LogisticRegression(..., penalty='l1')
params = {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]}
cv = 5
scoring = 'f1'

cv_lr_bankr_l1 = ...

In [ ]:
# На всей обучающей выборке обучите модель с L1 регуляризацией lr_bankr_l1
# Не забудьте зафиксировать RANDOM_STATE

lr_bankr_l1 = ...

In [ ]:
# Выведите количество отобранных признаков (признаков, регрессионный коэффициент при которых не равен 0) в lr_bankr_l1

...

In [ ]:
# Постройте отчет по метрикам классификации для модели lr_bankr_l1 на тестовой выборке

...

In [ ]:
# Посчитайте f1 для модели lr_bankr_l1 на тестовой выборке

...

In [ ]:
# Посчитайте AUC для модели lr_bankr_l1 на тестовой выборке

...